In [ ]:
# Necessary Libraries
import pandas as pd
import numpy as np
import networkx as nx
import json
import codecs
from collections import defaultdict
from pprint import pprint

In [ ]:
# Helper maps/dictionaries

# Map from string to its numerical weight
edge_weight_map = { '++': 2, '+': 1, '0': 0, '-': -1, '--': -2 }

# Data type of edge in graph
dt_edge = [('weight', int),
           ('weight_absolute', int), 
           ('strengthen', int), 
           ('weaken', int), 
           ('sign', int)]

# Maps string to dt_edge quintuple
def edge_map(w):
    w = edge_weight_map[w]
    abs_w = abs(w)
    sign = np.sign(w)
    return (w, abs_w, (abs_w if sign == 1 else 0), (abs_w if sign == -1 else 0), sign)

In [ ]:
# Extracts action systems from loaded data
def get_action_systems(data):
    entries = {}
    
    for i, entry in enumerate(data):
        entries[entry['id']] = entry['name']
    
    return entries

def get_life_entries(data, action_systems):
    identifier = {}
    labels = {}
    node_action_systems_id = {}
    node_action_systems = {}
    influences = {}
    
    for i, entry in enumerate(data):
        identifier[i] = entry['id']
        labels[i] = entry['title']
        node_action_systems_id[i] = entry['actionSystemId']
        node_action_systems[i] = action_systems[entry['actionSystemId']]
        influences[i] = entry['influence']
    
    return identifier, labels, node_action_systems_id, node_action_systems, influences

# Loads the json and creates a networkx multidi graph
def json_to_graph(path, encoding='utf-8', verbose=False):
    json_data = None
    action_systems = {}
    labels = {}
    node_action_systems = {}
    influences = {}
    
    with open(path, encoding=encoding) as data_file:
        json_data = json.load(data_file)
    
    # Print the loaded json data
    if verbose == True:
        ppriint(json_data)
    
    # Gets adjacency matrix and maps it to the dt_edge data type
    df = pd.DataFrame(json_data['connections'])
    df = df.applymap(edge_map)
    am = np.array(df, dtype=dt_edge)
    
    if verbose == True:
        # Print mapped adjacency matrix
        print(am)
    
    # Get action systems
    action_systems = get_action_systems(json_data['actionSystems'])
    
    # Get life entries
    identifier, labels, node_action_systems_id, node_action_systems, influences = get_life_entries(json_data['lifeEntries'], action_systems)
    
    # Create graph by loading the adjacency matrix
    G = nx.from_numpy_matrix(am, parallel_edges=False, create_using=nx.MultiDiGraph())
    
    # Set attributes on nodes
    nx.set_node_attributes(G, identifier, name='id')
    nx.set_node_attributes(G, labels, name='label')
    nx.set_node_attributes(G, node_action_systems_id, name='actionSystemId')
    nx.set_node_attributes(G, node_action_systems, name='actionSystem')
    nx.set_node_attributes(G, influences, name='influence')
    
    return G

def to_d3_json(G):
    nodes = []
    links = []
    
    labels = nx.get_node_attributes(G, name='label')
    influences = nx.get_node_attributes(G, name='influence')
    action_systems = nx.get_node_attributes(G, name='actionSystem')
    
    weights = nx.get_edge_attributes(G, name='weight')
    weight_absolute = nx.get_edge_attributes(G, name='weight_absolute')
    strengthen = nx.get_edge_attributes(G, name='strengthen')
    weaken = nx.get_edge_attributes(G, name='weaken')
    sign = nx.get_edge_attributes(G, name='sign')
    
    for i in range(len(labels)):
        nodes.append({
            'label': labels[i],
            'influence': influences[i],
            'actionSystem': action_systems[i]
        })
    
    print(weights)
    print(strengthen)

In [ ]:
def generate_metric(G, name, func, weights=[]):
    metric_data = {}
    if len(weights) == 0:
        # Metric is unweighted
        metric_data[name] = func(G)
    else:
        # Metric is weighted, go though every weight attribute
        for i, weight in enumerate(weights):
            metric_data['{}_{}'.format(name, weight)] = func(G, weight=weight)
            
    return metric_data

In [ ]:
def rebuild_metric(G, weights):
    metric_data = {}
    temp_data = {}
    for i, weight in enumerate(weights):
        temp_data['degree_{}'.format(weight)] = G.degree(weight=weight)
        temp_data['in_degree_{}'.format(weight)] = G.in_degree(weight=weight)
        temp_data['out_degree_{}'.format(weight)] = G.out_degree(weight=weight)
    
    temp_data['degree'] = G.degree()
    temp_data['in_degree'] = G.in_degree()
    temp_data['out_degree'] = G.out_degree()
    
    for i, metric in enumerate(temp_data):
        metric_data[metric] = {}
        for j, (node_id, value) in enumerate(temp_data[metric]):
            metric_data[metric][node_id] = value
    
    return metric_data

In [ ]:
def graph_to_json(G):

    json_object = {}
    nodes = []
    links = []
    
    # If a key does not exist on access, generate empty list
    # So every node gets a 'cycle' attribute, even if it's just an empty list
    cycles = defaultdict(list)
    
    # Get attributes from nodes (generated by JSON importer)
    identifier = nx.get_node_attributes(G, name='id')
    labels = nx.get_node_attributes(G, name='label')
    action_systems_id = nx.get_node_attributes(G, name='actionSystemId')
    action_systems = nx.get_node_attributes(G, name='actionSystem')
    influences = nx.get_node_attributes(G, name='influence')
    
    # Get all possible directed graph metrics and add them to 'node_metrics' dict
    node_metrics = {}
    degree_metrics = rebuild_metric(G, ['weight','weight_absolute','strengthen','weaken'])
    node_metrics.update(degree_metrics)
    degree_centrality = generate_metric(G, 'degree_centrality', nx.degree_centrality)
    node_metrics.update(degree_centrality)
    in_degree_centrality = generate_metric(G, 'in_degree_centrality', nx.in_degree_centrality)
    node_metrics.update(in_degree_centrality)
    out_degree_centrality = generate_metric(G, 'out_degree_centrality', nx.out_degree_centrality)
    node_metrics.update(out_degree_centrality)
    eigenvector_centrality_numpy = generate_metric(G, 'eigenvector_centrality_numpy', nx.eigenvector_centrality_numpy)
    node_metrics.update(eigenvector_centrality_numpy)
    eigenvector_centrality_numpy_weighted = generate_metric(G, 'eigenvector_centrality_numpy', nx.eigenvector_centrality_numpy, ['weight','weight_absolute','strengthen','weaken'])
    node_metrics.update(eigenvector_centrality_numpy_weighted)
    closeness_centrality = generate_metric(G, 'closeness_centrality', nx.degree_centrality)
    node_metrics.update(closeness_centrality)
    betweenness_centrality = generate_metric(G, 'betweenness_centrality', nx.betweenness_centrality)
    node_metrics.update(betweenness_centrality)
    betweenness_centrality_weighted = generate_metric(G, 'betweenness_centrality', nx.betweenness_centrality, ['weight','weight_absolute','strengthen','weaken'])
    node_metrics.update(betweenness_centrality_weighted)
    load_centrality = generate_metric(G, 'load_centrality', nx.load_centrality)
    node_metrics.update(load_centrality)
    load_centrality_weighted = generate_metric(G, 'load_centralityload_centrality', nx.load_centrality, ['weight_absolute','strengthen','weaken'])
    node_metrics.update(load_centrality_weighted)
    harmonic_centrality = generate_metric(G, 'harmonic_centrality', nx.harmonic_centrality)
    node_metrics.update(harmonic_centrality)
    betweenness_centrality_weighted = generate_metric(G, 'betweenness_centrality', nx.betweenness_centrality, ['weight','weight_absolute','strengthen','weaken'])
    node_metrics.update(betweenness_centrality_weighted)
    
    # Find cycles and build a 'cycle id' list
    # Nodes with the same 'cycle id' belong to the same cycle
    for i, cycle in enumerate(list(nx.simple_cycles(G))):
        for j, node in enumerate(cycle):
            cycles[node].append(i)
            
    # Generate sorted int list of 'cycle id'
    cycles_list = [int(i) for i in sorted(list(cycles.keys()))]
    
    # Fill 'nodes' dict
    for i in range(len(identifier)):
        attributes = {
            'id': identifier[i],
            'label': labels[i],
            'influence': influences[i],
            'actionSystemId': action_systems_id[i],
            'actionSystem': action_systems[i],
            'cycles': cycles[i]
        }
        
        for k, metric in enumerate(node_metrics):
            attributes[metric] = node_metrics[metric][i]
        
        nodes.append(attributes)

    # Get attributes from edges (generated by JSON importer)
    weight_absolute = nx.get_edge_attributes(G, name='weight_absolute')
    strengthen = nx.get_edge_attributes(G, name='strengthen')
    weaken = nx.get_edge_attributes(G, name='weaken')
    sign = nx.get_edge_attributes(G, name='sign')
    
    # Get all possible directed graph metrics and add them to 'edge_metrics' dict
    edge_metrics = {}
    edge_betweenness_centrality = generate_metric(G, 'edge_betweenness_centrality', nx.edge_betweenness_centrality)
    edge_metrics.update(edge_betweenness_centrality)
    edge_betweenness_centrality_weighted = generate_metric(G, 'edge_betweenness_centrality', nx.edge_betweenness_centrality, weights=['weight','weight_absolute','strengthen','weaken'])
    edge_metrics.update(edge_betweenness_centrality_weighted)
    edge_load_centrality = generate_metric(G, 'edge_load_centrality', nx.edge_load_centrality)
    edge_metrics.update(edge_load_centrality)
    
    # Fill 'links' dict
    for (from_node, to_node, weight) in G.edges(data='weight'):
        attributes = {
            'source': nodes[from_node]['id'],
            'target': nodes[to_node]['id'],
            'weight': weight,
            'weight_absolute': weight_absolute[(from_node, to_node, 0)],
            'strengthen': strengthen[(from_node, to_node, 0)],
            'weaken': weaken[(from_node, to_node, 0)],
            'sign': sign[(from_node, to_node, 0)]
        }
        
        for k, metric in enumerate(edge_metrics):
            attributes[metric] = edge_metrics[metric][(from_node, to_node)]
            
        links.append(attributes)
    
    json_object['cycles'] = cycles_list
    json_object['nodes'] = nodes
    json_object['links'] = links
    
    return json.JSONEncoder(ensure_ascii=False, allow_nan=False).encode(json_object)

In [ ]:
def graph_to_json_file(G, filename):
    file = codecs.open(filename, "w", "utf-8-sig")
    file.write(graph_to_json(G))
    file.close()

In [ ]:
G = json_to_graph('data/dataset_20180403/180331_Herbert_Heim.json')
to_d3_json(G)